In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models
from PIL import Image
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
!pip install -U albumentations

In [3]:
import albumentations as augment
from albumentations.pytorch import ToTensorV2
import numpy as np

In [4]:
# Load the trained model
def load_model(model_path, device="cpu"):
    model = models.mobilenet_v3_large(weights=None)  # Initialize MobileNetV3
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, 1)  # Adjust output classes
    model.load_state_dict(torch.load(model_path, map_location=device)) 
    print(model.classifier[3].weight)
    model.to(device)
    model.eval()
    return model


In [5]:
# Preprocess images
def preprocess_image(image_path):
    transform = augment.Compose([
        augment.Resize(height=370, width=370),
        augment.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return transform(image = image)["image"].unsqueeze(0).to(torch.float32)


In [6]:
# Perform inference
def classify_images(model, image_paths, device="cpu"):
    results = {}
    class_mapping = {1: "game", 0: "real"}
    # sigmoid = nn.Sigmoid()  # Sigmoid activation for binary classification
    for image_path in image_paths:
        image_tensor = preprocess_image(image_path).to(device)
        with torch.no_grad():
            output = model(image_tensor)  # output shape: (1,) or (1,1)
            probability = torch.sigmoid(output).cpu().numpy()  # ensure it's on CPU
            prediction = (probability > 0.5).astype(int)  # Convert probability to binary class
        
            print(f"{image_path}: Probability {probability}")
            print("Predicted class:", prediction)
        results[image_path] = prediction
    return results


In [7]:
# Function to display images with predictions
def display_images_with_predictions(predictions):
    for img_path, pred in predictions.items():
        img = Image.open(img_path)
        plt.figure()
        plt.imshow(img)
        plt.title(f"Predicted Class: {pred}")
        plt.axis("off")
        plt.show()

In [ ]:
model_path = "/kaggle/input/mobilenetv3large/pytorch/default/1/pretrained_MobileNet_V3_Large.pth"  # Path of model file
device = "cuda" if torch.cuda.is_available() else "cpu"

model = load_model(model_path, device)
  
# Directory containing images
image_dir = ["/kaggle/input/nilut-combined-nafnet/isp_nilut_output_09_images_nafnet", 
             "/kaggle/input/nilut-combined-nafnet/isp_nilut_output_10_images_nafnet"]

image_files = []
for dir_path in image_dir:
    image_files.extend(
        os.path.join(dir_path, f)
        for f in os.listdir(dir_path)
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    )

true_label = 0

predictions = classify_images(model, image_files, device)

correct = 0
total = len(predictions)

for img, pred in predictions.items():
    if pred == true_label:
        correct += 1

accuracy = correct / total if total > 0 else 0
print(f"\nAccuracy: {accuracy * 100:.2f}% ({correct}/{total} correct)")

# display_images_with_predictions(predictions)